In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-06-14 13:57:32.245813: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749909452.477297      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749909452.538823      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_datagen = ImageDataGenerator(rescale= 1./255, shear_range= 0.2, zoom_range= 0.2, horizontal_flip= True)

training_set = train_datagen.flow_from_directory('/kaggle/input/brain-tumor-detection/Brain_tumor_data/train',
                                                target_size= (224, 224), batch_size= 16, shuffle= True,
                                                class_mode= 'binary')

Found 215 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale= 1./255)
test_set = test_datagen.flow_from_directory('/kaggle/input/brain-tumor-detection/Brain_tumor_data/test',
                                           target_size= (224, 224), batch_size= 16, shuffle= False,
                                           class_mode= 'binary')

Found 32 images belonging to 2 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense

In [5]:
cnn = Sequential()

cnn.add(Conv2D(filters= 224, kernel_size= 3, activation= 'relu', input_shape= [224, 224, 3]))

cnn.add(MaxPool2D(pool_size= 2, strides= 2))
cnn.add(Conv2D(filters= 224, kernel_size= 3, activation= 'relu'))

cnn.add(MaxPool2D(pool_size= 2, strides= 2))
cnn.add(Dropout(0.5))
cnn.add(Flatten())
cnn.add(Dense(units= 128, activation= 'relu'))
cnn.add(Dense(units= 1, activation= 'sigmoid'))

cnn.compile(optimizer= 'Adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1749909468.537150      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749909468.537843      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [6]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 224)       │           6,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 224)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 224)       │         451,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 224)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 54, 54, 224)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 653184)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      83,607,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 84,065,889 (320.69 MB)

 Trainable params: 84,065,889 (320.69 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
cnn.fit(x= training_set, validation_data= test_set, epochs= 45)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/45


I0000 00:00:1749909473.257328      76 service.cc:148] XLA service 0x7924d0005fa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749909473.258397      76 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749909473.258420      76 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749909473.591873      76 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/14 ━━━━━━━━━━━━━━━━━━━━ 3:28 16s/step - accuracy: 0.5625 - loss: 0.6946

I0000 00:00:1749909486.631166      76 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/14 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.5225 - loss: 4.9384

E0000 00:00:1749909492.852829      76 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1749909493.121034      76 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


14/14 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.5249 - loss: 4.4646 - val_accuracy: 0.6562 - val_loss: 0.6543
Epoch 2/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 215ms/step - accuracy: 0.6324 - loss: 0.6421 - val_accuracy: 0.5000 - val_loss: 0.6852
Epoch 3/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 199ms/step - accuracy: 0.6765 - loss: 0.5960 - val_accuracy: 0.6250 - val_loss: 0.6235
Epoch 4/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 198ms/step - accuracy: 0.6520 - loss: 0.6775 - val_accuracy: 0.6250 - val_loss: 0.6349
Epoch 5/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 212ms/step - accuracy: 0.6894 - loss: 0.6043 - val_accuracy: 0.6875 - val_loss: 0.5401
Epoch 6/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 221ms/step - accuracy: 0.7673 - loss: 0.5024 - val_accuracy: 0.6562 - val_loss: 0.5462
Epoch 7/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.7836 - loss: 0.5136 - val_accuracy: 0.6875 - val_loss: 0.5432
Epoch 8/45
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 200ms/step - accuracy: 0.7179 - loss: 0.5760 - val_accuracy: 0.5938 - val_lo

In [8]:
from keras.preprocessing import image
from tensorflow.keras.utils import load_img, img_to_array

In [9]:
import os

# Path to the directory containing the images you want to predict
image_directory = '/kaggle/input/brain-tumor-detection/Brain_tumor_data/prediction/'

# Get a list of all image files in the directory
image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

print(f"Found {len(image_files)} images in {image_directory}\n")

for image_name in image_files:
    image_path = os.path.join(image_directory, image_name)
    print(f"Processing image: {image_name}")

    # Load the image
    test_image = load_img(image_path, target_size=(224, 224))

    # Convert to NumPy array
    test_image = img_to_array(test_image)

    # Expand dimensions to create a batch of 1 image
    test_image = np.expand_dims(test_image, axis=0)

    test_image = test_image / 255.0

    # Make prediction
    result = cnn.predict(test_image)
    training_set.class_indices

    # Print raw prediction result (probabilities or logits)
    print(f"Raw prediction for {image_name}: {result}")

    # Interpret the result
    if result.shape[1] == 1: # Binary classification with sigmoid output
        if result[0][0] > 0.5:
            prediction_label = training_set.class_indices.get(1, "Yes, Tumor is detected") 
            confidence = result[0][0] * 100
        else:
            prediction_label = training_set.class_indices.get(0, "No Tumor. You are Healthy")
            confidence = (1 - result[0][0]) * 100
    
    print(f"Prediction: {prediction_label} (Confidence: {confidence:.2f}%)\n")

print("All images processed.")

Found 6 images in /kaggle/input/brain-tumor-detection/Brain_tumor_data/prediction/

Processing image: yes1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Raw prediction for yes1.jpg: [[0.98592365]]
Prediction: Yes, Tumor is detected (Confidence: 98.59%)

Processing image: no2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Raw prediction for no2.jpg: [[0.03380277]]
Prediction: No Tumor. You are Healthy (Confidence: 96.62%)

Processing image: yes2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Raw prediction for yes2.jpg: [[0.9865767]]
Prediction: Yes, Tumor is detected (Confidence: 98.66%)

Processing image: no3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Raw prediction for no3.jpg: [[0.00130246]]
Prediction: No Tumor. You are Healthy (Confidence: 99.87%)

Processing image: yes3.JPG
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Raw prediction for yes3.JPG: [[0.91661173]]
Prediction: Yes, Tumor is detected (Confidence: 91.66%)

Processing image: no1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Raw prediction for no1.